In [38]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.fftpack import fft2, ifft2
from scipy.integrate import odeint, solve_ivp
import time

# Parameters
L = 10
n = 64
nu = 0.001
amp = 1
Lx, Ly = 20, 20
nx, ny = 64, 64
N = nx * ny

# # Spatial domain & initial conditions
# x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
# x = x2[:nx]
# y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
# y = y2[:ny]
# X, Y = np.meshgrid(x, y)
# w = np.exp(-X**2 - Y**2 / 20).flatten()  # Initialize as complex "+ 1j * np.zeros((nx, ny))"
# tspan = np.arange(0, 4.5, 0.5)

# # Spectral & Initial Conditions
# kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
# kx[0] = 1e-6
# ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
# ky[0] = 1e-6
# KX, KY = np.meshgrid(kx, ky)
# K = KX**2 + KY**2



In [40]:
# HW 4
m = n**2
dx = (2 * L) / n
dy = (2 * L) / n


# Matrix A
e0 = np.zeros((m, 1))  # vector of zeros
e1 = np.ones((m, 1))   # vector of ones
e2 = np.copy(e1)    # copy the one vector
e4 = np.copy(e0)    # copy the zero vector

for j in range(1, n+1):
    e2[n*j-1] = 0  # overwrite every n^th value with zero
    e4[n*j-1] = 1  # overwirte every n^th value with one

# Shift to correct positions
e3 = np.zeros_like(e2)
e3[1:m] = e2[0:m-1]
e3[0] = e2[m-1]

e5 = np.zeros_like(e4)
e5[1:m] = e4[0:m-1]
e5[0] = e4[m-1]

# Place diagonal elements
diagonals = [e1.flatten(), e1.flatten(), e5.flatten(), 
             e2.flatten(), -4 * e1.flatten(), e3.flatten(), 
             e4.flatten(), e1.flatten(), e1.flatten()]
offsets = [-(m-n), -n, -n+1, -1, 0, 1, n-1, n, (m-n)]

A = spdiags(diagonals, offsets, m, m).toarray()
A = A / (dx**2)
A[0 , 0] = 2 / (20/64) ** 2 

# Matrix B
e1 = np.ones((m, 1))   # vector of ones
e2 = np.ones((m, 1))   # vector of ones
e2 = e2 * -1

diagonals = [e1.flatten(), e2.flatten(), e1.flatten(), e2.flatten()]
offsets = [-(m-n), -n, n, (m-n)]

B = spdiags(diagonals, offsets, m, m).toarray()
B = B /(2*dx)

# Matrix C
e1 = np.ones((m, 1))   # vector of ones
e2 = np.ones((m, 1))   # vector of ones
e2 = e2 * -1

diagonals = [e2.flatten(), e1.flatten()]
offsets = [-1, 1]

C = spdiags(diagonals, offsets, m, m).toarray()

for i in range(n):
    C[(i+1)*n - 1, i*n] = 1
    C[i*n, (i+1)*n - 1] = -1
    C[i*n, i*n-1] = 0
    C[i*n-1, i*n] = 0

C = C /(2*dy)



In [41]:

# Spatial domain & initial conditions
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)
w = np.exp(-X**2 - Y**2 / 20).flatten()  # Initialize as complex "+ 1j * np.zeros((nx, ny))"
tspan = np.arange(0, 4.5, 0.5)

# Spectral & Initial Conditions
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

In [44]:
def RHS(t, omega):
    omega_matrix = omega.reshape((nx, ny))
    wt = fft2(omega_matrix)
    psit = -wt/K
    psi = np.real(ifft2(psit)).flatten()
    
    return nu * (A.dot(omega)) - (B.dot(psi))*(C.dot(omega)) + (C.dot(psi))*(B.dot(omega))

start_time = time.time() # Record start time

sol = solve_ivp(RHS, [0, 4], w, t_eval=tspan, method='RK45')

end_time = time.time() # Record end time
elapsed_time = end_time - start_time

print(sol.y)
print(f"Elapsed time: {elapsed_time:.2f} seconds")

A1 = sol.y

[[2.50656748e-46 3.59343426e-45 1.89151801e-44 ... 1.90842651e-42
  4.82342450e-42 1.14736334e-41]
 [1.17762859e-43 6.53684735e-43 2.62032800e-42 ... 1.58954416e-40
  3.55995327e-40 7.45538537e-40]
 [4.55107657e-41 1.93087628e-40 6.43608612e-40 ... 2.49019716e-38
  5.07450761e-38 9.69262315e-38]
 ...
 [1.96785570e-38 1.23636238e-37 5.46949973e-37 ... 4.35894300e-35
  1.02319688e-34 2.22797769e-34]
 [6.19028421e-41 5.34124949e-40 2.93842095e-39 ... 3.68802103e-37
  9.45257062e-37 2.23317639e-36]
 [1.60178709e-43 1.99879166e-42 1.41262426e-41 ... 2.80602636e-39
  7.96346160e-39 2.07361555e-38]]
Elapsed time: 0.98 seconds


In [63]:
# Part B ---------------------------------------------------------------------------------------

# A/b
start_time = time.time()

def direct_solver(t, omega):     
    psi = np.linalg.solve(A, omega).flatten()

    rhs = nu * (A.dot(omega)) - (B.dot(psi))*(C.dot(omega)) + (C.dot(psi))*(B.dot(omega))
    return rhs.flatten()

sol = solve_ivp(direct_solver, [0, 4], w, t_eval=tspan, method='RK45')
A2 = sol.y

end_time = time.time() 

print(sol.y)
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

[[ 2.50656748e-46  1.17762859e-43  4.55107657e-41 ...  1.96785570e-38
   6.19028421e-41  1.60178709e-43]
 [-1.78630705e-36 -2.26093387e-29 -4.24661881e-29 ...  7.20814729e-28
   1.69117621e-28  5.05834665e-29]
 [ 1.02614869e-36  1.85614646e-29  3.48720993e-29 ... -6.58759786e-28
  -1.49205998e-28 -4.38046443e-29]
 ...
 [-5.75086257e-26 -1.48306461e-19 -2.20165937e-20 ...  4.01674620e-20
   3.00460834e-20  2.47633369e-20]
 [-6.68013806e-25 -7.82424220e-19 -2.91637720e-19 ...  1.34371413e-19
   1.76009486e-19  1.15181522e-20]
 [ 3.02201851e-24  1.67449651e-18  1.23961335e-18 ... -8.94163775e-19
  -9.80736832e-19  3.12761254e-20]]
Elapsed time: 56.72 seconds


In [64]:
# LU
from scipy.linalg import lu, solve_triangular, lu_factor, lu_solve

start_time = time.time()

def LU_solver(t, omega):
    # P, L, U = lu(A)
    # Pw = np.dot(P, w)
    # y = solve_triangular(L, Pw, lower=True)
    # psi = solve_triangular(U, y)
    
    LU, piv = lu_factor(A)
    psi = lu_solve((LU, piv), omega)  
    
    rhs = nu * (A.dot(omega)) - (B.dot(psi))*(C.dot(omega)) + (C.dot(psi))*(B.dot(omega))
    return rhs.flatten()

sol = solve_ivp(LU_solver, [0, 4], w, t_eval=tspan, method='RK45')
A3 = sol.y

end_time = time.time() 

print(A3)

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

[[ 2.50656748e-46  1.17762859e-43  4.55107657e-41 ...  1.96785570e-38
   6.19028421e-41  1.60178709e-43]
 [-1.78630705e-36 -2.26093387e-29 -4.24661881e-29 ...  7.20814729e-28
   1.69117621e-28  5.05834665e-29]
 [ 1.02614869e-36  1.85614646e-29  3.48720993e-29 ... -6.58759786e-28
  -1.49205998e-28 -4.38046443e-29]
 ...
 [-5.75086257e-26 -1.48306461e-19 -2.20165937e-20 ...  4.01674620e-20
   3.00460834e-20  2.47633369e-20]
 [-6.68013806e-25 -7.82424220e-19 -2.91637720e-19 ...  1.34371413e-19
   1.76009486e-19  1.15181522e-20]
 [ 3.02201851e-24  1.67449651e-18  1.23961335e-18 ... -8.94163775e-19
  -9.80736832e-19  3.12761254e-20]]
Elapsed time: 61.50 seconds


In [65]:
# # BICGSTAB
# from scipy.sparse.linalg import bicgstab

# start_time = time.time()
# psi, info = bicgstab(A, w)


# BIC_ans = nu * (A.dot(w)) + (B.dot(psi))*(C.dot(w)) - (C.dot(psi))*(B.dot(w))
# BIC_ans = BIC_ans.flatten()
# # A4 = BIC_ans
# print(BIC_ans)

# end_time = time.time() 
# elapsed_time = end_time - start_time
# print(f"Elapsed time: {elapsed_time:.2f} seconds")

In [66]:
# # GMRES
# from scipy.sparse.linalg import gmres

# start_time = time.time()
# psi, info = gmres(A, w)

# GMR_ans = nu * (A.dot(w)) + (B.dot(psi))*(C.dot(w)) - (C.dot(psi))*(B.dot(w))
# GMR_ans = GMR_ans.flatten()
# # A5 = GMR_ans
# print(GMR_ans)

# end_time = time.time() 
# elapsed_time = end_time - start_time
# print(f"Elapsed time: {elapsed_time:.2f} seconds")